# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe
dim_experience = pd.DataFrame(fct_guest_spending['park_experience_type'].unique(),
                              columns=['park_experience_type']
                             ).reset_index(drop=True)

df1 = fct_guest_spending[(fct_guest_spending['visit_date'] >= '2024-07-01') 
  & (fct_guest_spending['visit_date'] < '2024-07-31')]
df2 = df1.groupby('park_experience_type').agg(
  amount_spent =('amount_spent','sum'),
  guest_count =('guest_id','count')
).reset_index()
df2['avg_spend'] = df2['amount_spent']/df2['guest_count']
df3 = dim_experience.merge(df2, on = 'park_experience_type', 
                           how = 'left').fillna(0).drop(columns=['amount_spent','guest_count'])
df3

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
df1 = fct_guest_spending[(fct_guest_spending['visit_date'] >= '2024-08-01') 
    & (fct_guest_spending['visit_date'] < '2024-08-31')].sort_values (
  by = ['guest_id','visit_date'], ascending = [True, True])
multi_visit = df1.groupby('guest_id').filter(lambda x: len(x)>1)
#multi_visit['first_date'] = multi_visit.groupby('guest_id')['visit_date'].transform('min')
#multi_visit['last_date'] = multi_visit.groupby('guest_id')['visit_date'].transform('max')
summary_df = multi_visit.groupby('guest_id').agg(
  first_date = ('visit_date','first'),
  last_date = ('visit_date','last'),
  first_spending = ('amount_spent', lambda x: x.iloc[0]),
  last_spending = ('amount_spent', lambda x: x.iloc[-1])
).reset_index()
summary_df['diff'] = summary_df['first_spending'] -summary_df['last_spending'] 
summary_df

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
df1 = fct_guest_spending [(fct_guest_spending['visit_date'] >= '2024-09-01') 
    & (fct_guest_spending['visit_date'] < '2024-09-30')].reset_index()

# df2 = df1.groupby('guest_id', as_index = False)['amount_spent'].sum().reset_index(drop=True)
df2 = df1.groupby('guest_id')['amount_spent'].sum()\
         .reset_index()\
         .loc[lambda x: x['amount_spent'] > 0]
# df2 = df2.query('amount_spent' > 0)
# df2 = fct_guest_spending.groupby('guest_id')['amount_spent'].sum().reset_index()
df2['categorization'] = pd.cut(
  df2['amount_spent'],
  bins = [0, 50, 100, np.inf],
  labels = ['Low','Medium','High'],
  right = False 
)
# df3 = df1.merge(df2,on = 'guest_id', how = 'left')
df2

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)